In [1]:
import pandas as pd

In [8]:
df = pd.read_csv("~/real_estate/data/unsupervised_dfs/df_20181210.csv")

In [12]:
features = [
 'median_sqft_value',
 'percent_decreasing',
 'percent_increasing',
 'zhvi_singlefam',
 'zhvi_1bed',
 'zhvi_2bed',
 'zhvi_3bed',
 'zhvi_condo',
 '90_day_treasury_bill_rate',
 'mean_hourly_earnings_adj',
 'mean_fixed_mortgage_rate',
 'total_nonfarm_payroll',
 'zhvi_sqft_value']

target = "zhvi_sqft_value"

In [13]:
# Store data from previous, future months and years
feature_mo = {}
feature_yr = {}
zhvi_sqft_value_12_months = []

for i in features:
    feature_mo[i+"_mo"] = []
    feature_yr[i+"_yr"] = []

for i in df.index:
    
    current_row = df.iloc[i]
    region_name = current_row["RegionName"]
    month = int(current_row["month"])
    year = int(current_row["year"])
    next_year = year + 1
    
    # Get column values of previous time preiods
    last_year = year - 1
    if month == 1:
        last_month_month = 12
        last_month_year = year - 1
    else:
        last_month_month = month - 1
        last_month_year = year
    
    last_month_row = df.loc[(df["RegionName"] == region_name) & 
                            (df["month"] == last_month_month) &
                            (df["year"] == last_month_year)]
    
    last_year_row = df.loc[(df["RegionName"] == region_name) &
                           (df["month"] == month) &
                           (df["year"] == last_year)]
    
    next_year_row = df.loc[(df["RegionName"] == region_name) &
                           (df["month"] == month) &
                           (df["year"] == next_year)]
    
    for i in features:
        
        feature_mo[i+"_mo"].append(last_month_row[i].values)
        feature_yr[i+"_yr"].append(last_year_row[i].values)
        
    next_year_target = next_year_row["zhvi_sqft_value"]
    
    if len(next_year_target) == 0:
        next_year_target = None
    else:
        next_year_target = next_year_target.values[0]
    
    zhvi_sqft_value_12_months.append(next_year_target)
    
# Add data to dataframe
for i in feature_mo.keys():
    new_list = [j[0] if len(j) > 0 else None for j in feature_mo[i]]
    feature_mo[i] = new_list

for i in feature_yr.keys():
    new_list = [j[0] if len(j) > 0 else None for j in feature_yr[i]]
    feature_yr[i] = new_list
    
for i in feature_mo.keys():
    df[i] = feature_mo[i]
    
for i in feature_yr.keys():
    df[i] = feature_yr[i]
    
df["zhvi_sqft_value_12_months"] = zhvi_sqft_value_12_months

In [14]:
# Filter null values
df = df.dropna()

In [15]:
# Export
df.to_csv("~/real_estate/data/supervised_dfs/df_20181210.csv", index=False)

In [16]:
df.head()

,RegionName,median_sqft_value,percent_decreasing,percent_increasing,zhvi_1bed,zhvi_2bed,zhvi_3bed,zhvi_sqft_value,zhvi_condo,zhvi_singlefam,...,90_day_treasury_bill_rate_yr,median_sqft_value_yr,zhvi_1bed_yr,mean_hourly_earnings_adj_yr,zhvi_condo_yr,percent_decreasing_yr,percent_increasing_yr,zhvi_2bed_yr,zhvi_sqft_value_yr,zhvi_sqft_value_12_months
11798,94109,682.0,54.79,35.10,481300.0,746200.0,1190200.0,3.196,674600.0,2242600.0,...,0.14,677.0,512600.0,22.74,689200.0,60.67,28.87,768100.0,3.176,3.290
11799,20002,308.0,28.67,54.74,290500.0,346000.0,368400.0,1.824,296600.0,376900.0,...,0.14,305.0,286400.0,22.74,296400.0,41.97,44.82,343500.0,1.694,1.894
11800,90046,479.0,57.95,30.91,307300.0,634300.0,930600.0,2.298,405700.0,945700.0,...,0.14,501.0,341900.0,22.74,440500.0,70.76,20.34,652300.0,2.268,2.304
11801,20009,503.0,40.72,37.54,331900.0,512700.0,681600.0,3.048,388900.0,714100.0,...,0.14,505.0,344600.0,22.74,389300.0,33.92,48.06,508900.0,2.782,3.068
11802,7030,463.0,69.90,17.66,330500.0,477500.0,901300.0,2.656,446100.0,951000.0,...,0.14,471.0,341600.0,22.74,442300.0,68.06,18.35,485200.0,2.712,2.842
